In [8]:
%load_ext sql
%sql sqlite:///complaint.db
import time
%sql drop index if exists helpful_index;
%sql analyze

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Done.
Done.


[]

Activity 13: Indexes
===========

Let's play with the [consumer complaint database](https://catalog.data.gov/dataset/consumer-complaint-database) from data.gov

In [9]:
%sql select count(*) from complaints;

Done.


count(*)
471871


In [10]:
%sql select * from complaints limit 5;

Done.


Date_received,Product,Subproduct,Issue,Subissue,Consumer_narrative,Company_public_response,Company,State,ZIP_code,Submitted_via,Date_sent_to_company,Company_response,Timely_response,Consumer_disputed,Complaint_ID
10/28/2015,Debt collection,"Other (phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,,Company believes it acted appropriately as authorized by contract or law,"First National Collection Bureau, Inc.",CO,810XX,Web,10/28/2015,Closed with explanation,Yes,,1629130
10/28/2015,Credit reporting,,Incorrect information on credit report,Public record,,Company chooses not to provide a public response,"FNIS (Fidelity National Information Services, Inc.)",MI,48334,Web,10/28/2015,Closed with explanation,Yes,,1628785
10/28/2015,Money transfers,International money transfer,Fraud or scam,,,,Navy FCU,VA,221XX,Web,10/28/2015,In progress,Yes,,1629655
10/28/2015,Debt collection,,Cont'd attempts collect debt not owed,Debt is not mine,,,"EOS Holdings, Inc.",FL,321XX,Web,10/28/2015,Closed with explanation,Yes,Yes,1629306
10/28/2015,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt was paid,,,Transworld Systems Inc.,CA,961XX,Web,10/28/2015,In progress,Yes,,1629407


### Exercise 1: Query without an index

First, let's start off by writing a query to find the **counts of the top 5 Product, State pairs** in the complaints database (return the product and state as well as the count).  Use the single-line syntax for simple timing so we can see how long the query takes:

In [11]:
%time %sql SELECT product, state, count(*) AS c FROM complaints GROUP BY product, state ORDER BY c DESC LIMIT 5;

Done.
CPU times: user 791 ms, sys: 78.3 ms, total: 869 ms
Wall time: 888 ms


Product,State,c
Mortgage,CA,29434
Mortgage,FL,18659
Debt collection,CA,11066
Mortgage,NY,10002
Credit reporting,CA,9402


### Exerise 2: Single search key index

Now create a _single-key_ index such that the above query is faster!  The syntax to create an index in SQL is:
> DROP INDEX IF EXISTS index_name;
> CREATE INDEX index_name ON table(attributes);

In [12]:
%%sql
DROP INDEX IF EXISTS state_index;
CREATE INDEX state_index ON complaints(state);

Done.
Done.


[]

In [13]:
%time %sql SELECT product, state, count(*) AS c FROM complaints GROUP BY product, state ORDER BY c DESC LIMIT 5;

Done.
CPU times: user 791 ms, sys: 71.9 ms, total: 862 ms
Wall time: 879 ms


Product,State,c
Mortgage,CA,29434
Mortgage,FL,18659
Debt collection,CA,11066
Mortgage,NY,10002
Credit reporting,CA,9402


### Exercise 3

Now, create a _covering_ index for the query and then see how long it takes to run!

In [14]:
%%sql
DROP INDEX IF EXISTS state_product_index;
CREATE INDEX state_product_index ON complaints(state, product);

Done.
Done.


[]

In [15]:
%time %sql SELECT product, state, count(*) AS c FROM complaints GROUP BY product, state ORDER BY c DESC LIMIT 5;

Done.
CPU times: user 110 ms, sys: 6.58 ms, total: 117 ms
Wall time: 116 ms


Product,State,c
Mortgage,CA,29434
Mortgage,FL,18659
Debt collection,CA,11066
Mortgage,NY,10002
Credit reporting,CA,9402


### Exercise 3(b):

Use EXPLAIN to see if sqlite used/recognized your covering index.  EXPLAIN is an operator that tells SQL to explain its query plan... we'll look into this in more depth later.  For now, the syntax is:
> EXPLAIN QUERY PLAN your_query_here;

In [16]:
%%sql
EXPLAIN QUERY PLAN
SELECT product, state, count(*) AS c
FROM complaints
GROUP BY product, state ORDER BY c DESC LIMIT 5;

Done.


selectid,order,from,detail
0,0,0,SCAN TABLE complaints USING COVERING INDEX state_product_index
0,0,0,USE TEMP B-TREE FOR ORDER BY
